In [1]:
import dask.array as da
from dask.distributed import Client
import os
import sys
import random
import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image
from sklearn import preprocessing
from sklearn.preprocessing import scale
import tensorflow as tf
from tensorflow import keras

In [2]:
N_train = [2500, 2500, 2500, 2500, 2500, 2500]
N_test_start = [2500, 2500, 2500, 2500, 2500, 2500]
N_test = [250, 250, 250, 250, 250, 250]
N_class = len(N_train)
data_sizeX = 200
data_sizeY = 600
mask_sizeX = data_sizeX
mask_sizeY = data_sizeY
num_channel = 1
train_data1 = []
train_data2 = []
train_mask = []
test_data1 = []
test_data2 = []
test_mask = []
path = 'C:/Users/Tacab/Desktop/derin_ogrenme_gpr/' 
sub_path = 'expa/'

In [3]:
for i in range(N_class):
    for j in range(N_train[i]):
        data1 = sio.loadmat(path + 'dataset/%d/data1/%d.mat' % (i + 1, j + 1))['data1']
        resized_data1 = np.array(Image.fromarray(data1).resize((data_sizeY, data_sizeX)))
        train_data1.append(resized_data1.reshape((data_sizeX, data_sizeY, 1)))

        data2 = sio.loadmat(path + 'dataset/%d/data2/%d.mat' % (i + 1, j + 1))['data2']
        resized_data2 = np.array(Image.fromarray(data2).resize((data_sizeY, data_sizeX)))
        train_data2.append(resized_data2.reshape((data_sizeX, data_sizeY, 1)))

        mask = sio.loadmat(path + 'dataset/%d/mask/%d.mat' % (i + 1, j + 1))['mask']
        resized_mask = np.array(Image.fromarray(mask).resize((mask_sizeY, mask_sizeX)))
        train_mask.append(resized_mask.reshape((mask_sizeX, mask_sizeY, 1)))

    for j in range(N_test_start[i], N_test_start[i] + N_test[i]):
        data1 = sio.loadmat(path + 'dataset/%d/data1/%d.mat' % (i + 1, j + 1))['data1']
        resized_data1 = np.array(Image.fromarray(data1).resize((data_sizeY, data_sizeX)))
        test_data1.append(resized_data1.reshape((data_sizeX, data_sizeY, 1)))

        data2 = sio.loadmat(path + 'dataset/%d/data2/%d.mat' % (i + 1, j + 1))['data2']
        resized_data2 = np.array(Image.fromarray(data2).resize((data_sizeY, data_sizeX)))
        test_data2.append(resized_data2.reshape((data_sizeX, data_sizeY, 1)))

        mask = sio.loadmat(path + 'dataset/%d/mask/%d.mat' % (i + 1, j + 1))['mask']
        resized_mask = np.array(Image.fromarray(mask).resize((mask_sizeY, mask_sizeX)))
        test_mask.append(resized_mask.reshape((mask_sizeX, mask_sizeY, 1)))


In [4]:
train_data1 = da.from_array(np.array(train_data1), chunks=(1, data_sizeX, data_sizeY, num_channel))
train_data2 = da.from_array(np.array(train_data2), chunks=(1, data_sizeX, data_sizeY, num_channel))
train_mask = da.from_array(np.array(train_mask), chunks=(1, mask_sizeX, mask_sizeY, 1))
test_data1 = da.from_array(np.array(test_data1), chunks=(1, data_sizeX, data_sizeY, num_channel))
test_data2 = da.from_array(np.array(test_data2), chunks=(1, data_sizeX, data_sizeY, num_channel))
test_mask = da.from_array(np.array(test_mask), chunks=(1, mask_sizeX, mask_sizeY, 1))

In [5]:
def diffusion_model():
    inputs1 = keras.layers.Input(shape=(data_sizeX, data_sizeY, num_channel))
    inputs2 = keras.layers.Input(shape=(data_sizeX, data_sizeY, num_channel))

    # Diffusion process
    diff1 = keras.layers.Conv2D(16, (3, 3), padding='same', activation='relu')(inputs1)
    diff1 = keras.layers.Conv2D(16, (3, 3), padding='same', activation='relu')(diff1)
    diff2 = keras.layers.Conv2D(16, (3, 3), padding='same', activation='relu')(inputs2)
    diff2 = keras.layers.Conv2D(16, (3, 3), padding='same', activation='relu')(diff2)

    # Fusion
    fuse = keras.layers.Add()([diff1, diff2])
    outputs = keras.layers.Conv2D(1, (3, 3), padding='same', activation='sigmoid')(fuse)

    model = keras.models.Model(inputs=[inputs1, inputs2], outputs=outputs)
    return model

In [8]:
epochs = 1
batch_size = 1

In [9]:
model = diffusion_model()

model.compile(optimizer='adam', loss='mean_squared_error')


model.fit([train_data1, train_data2], train_mask, epochs=epochs, batch_size=batch_size)


test_loss = model.evaluate([test_data1, test_data2], test_mask)

print("Test Loss:", test_loss)

Epoch 1/4
1875/1875 [==============================] - 4118s 2s/step - loss: 1113.6929
Epoch 2/4
1875/1875 [==============================] - 4297s 2s/step - loss: 1113.6616
Epoch 3/4
1875/1875 [==============================] - 5393s 3s/step - loss: 1113.6534
Epoch 4/4
47/47 [==============================] - 81s 2s/step - loss: 1113.6819
Test Loss: 1113.681884765625


In [10]:
model_path = path + sub_path
model.save_weights(model_path)

In [11]:
test_pred = model.predict([test_data1, test_data2])

47/47 [==============================] - 83s 2s/step


In [13]:
test_data1_np = test_data1.compute()
test_data2_np = test_data2.compute()
test_mask_np = test_mask.compute()

In [14]:
sio.savemat(path + sub_path + 'test_data_dask.mat', {'test_data1': test_data1_np})
sio.savemat(path + sub_path + 'test_data_dask.mat', {'test_data2': test_data2_np})
sio.savemat(path + sub_path + 'test_mask_dask.mat', {'test_mask': test_mask_np})
sio.savemat(path + sub_path + 'test_pred_dask.mat', {'test_pred': test_pred})